## Target analysis of TA of ideal WL-PSI

### Inspect data

In [ ]:
from cycler import cycler
from glotaran.io import load_parameters, save_result
from glotaran.optimization.optimize import optimize
from glotaran.project.scheme import Scheme
from pyglotaran_extras.plotting.plot_overview import plot_overview
from pyglotaran_extras.plotting.plot_traces import (
    plot_fitted_traces,
    select_plot_wavelengths,
)
from pyglotaran_extras.inspect import show_a_matrixes

In [ ]:
from pyglotaran_extras import plot_data_overview

# DATA_PATH1 = "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_reva.ascii"
# DATA_PATH2 = "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revb.ascii"
DATA_PATH3 = "data/synWTred_idealc.ascii"
DATA_PATH4 = "data/synWTred_ideald.ascii"
fig, axes = plot_data_overview(
    DATA_PATH3, nr_of_data_svd_vectors=5, linlog=False
)
# change color map seismic or bwr
# axes[0].set_cmap('seismic')

In [ ]:
from pyglotaran_extras import plot_data_overview

# DATA_PATH1 = "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_reva.ascii"
# DATA_PATH2 = "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revb.ascii"
DATA_PATH3 = "data/synWTred_idealc.ascii"
DATA_PATH4 = "data/synWTred_ideald.ascii"
fig, axes = plot_data_overview(
    DATA_PATH4, nr_of_data_svd_vectors=5, linlog=True, linthresh=10
)
# change color map seismic or bwr
# axes[0].set_cmap('seismic')

## Target Analysis

### Used model and parameters

In [ ]:
target_model_path = "models/20230824model_PSI_TA_ideal.yml"

In [ ]:
target_parameters_path = "models/20230824optimized_parameters_ideal.csv"
optimizedparameters = load_parameters(target_parameters_path)

#### Model file

In [ ]:
# Uncomment the following 2 lines to display the target model file in the notebook
# from glotaran.utils.ipython import display_file
# display_file(target_model_path, syntax="yaml")

# Alternatively (recommended), open the file in a text editor to see the model definition

#### Parameters file

In [ ]:
# Uncomment the next line and run the cell to print the starting values of the analysis
# These starting values have already been optimized, hence the name optimizedparameters

# optimizedparameters

### Create scheme and optimize it

In [ ]:
target_scheme = Scheme(
    model=target_model_path,  # type: ignore
    parameters=optimizedparameters,
    maximum_number_function_evaluations=7,
    clp_link_tolerance=0.1,
    data={
        # TA data
        # "670TR1": DATA_PATH1,
        # "670TR2": DATA_PATH2,
        "700TR1": DATA_PATH3,
        "700TR2": DATA_PATH4,
        # "Red1SADS": "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_reve.ascii",
        # "WLRCSADS": "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revf.ascii",
        # "Red2SADS": "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revg.ascii",
        # "WLRP1SADS": "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revh.ascii",
    },  # type: ignore
)
target_scheme.validate()

-  15         7.7842e+02 with Red 2
-   7         8.4722e+02 without Red 2
-   7         8.3635e+02 doas par free
-    5         8.1524e+02  reduced RP1 weight
-    5         8.0521e+02 more reduced RP1 & RC weight

In [ ]:
target_result = optimize(target_scheme, raise_exception=True)

To save the results of the optimization we can use the `save_result` command.

Because it saves *everything* it consumes about 50MB of disk space per save.

In [ ]:
save_result(
    result=target_result,
    result_path="results/20230824/result.yaml",
    allow_overwrite=True,
)

### Results and parameters

In [ ]:
# Just call the result to get the optimization result summary.
target_result
# For easier copy-and-paste try:
# print(target_result)

In [ ]:
# Just call the result to get the optimization result summary.
target_result
# For easier copy-and-paste try:
# print(target_result)

In [ ]:
# Access the result's `optimized_parameters` to print a markdown table of the optimized parameters:
target_result.optimized_parameters

# Result plots

<sub>Note: The color scheme of the plots in this notebook may not match published figures.</sub>

## Fit quality

In [ ]:
target_result_TA = (
    # target_result.data["670TR1"],
    # target_result.data["670TR2"],
    target_result.data["700TR1"],
    target_result.data["700TR2"],
)
wavelengths = select_plot_wavelengths(target_result_TA, equidistant_wavelengths=True)
plot_fitted_traces(target_result_TA, wavelengths, linlog=True, linthresh=1);

The above command `plot_fitted_traces` is used to plot a selection of traces for a set of wavelengths (autogenerated using the `select_plot_wavelengths` function).
To show to make a manual selection of traces, and 'dress up the plot' see the code below, which reproduces Figure 2 of the paper.

In [ ]:
# Reproduction of Figure 2 of the paper
import warnings
from pyglotaran_extras.plotting.style import ColorCode as cc

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fig, ax_ = plot_fitted_traces(
        target_result_TA,
        [685, 700, 720, 760],
        linlog=True,
        linthresh=1,  # published figure uses 0.3 for easthetic reasons, but here 1 looks better
        axes_shape=(2, 2),
        figsize=(6, 4),
        title="",
        per_axis_legend=True,
        cycler=cycler(
            color=[
                cc.grey,
                cc.black,
                cc.grey,
                cc.black,
                cc.orange,
                cc.red,
                cc.orange,
                cc.red,
            ]
        ),
    )
    handles, labels = ax_.flatten()[0].get_legend_handles_labels()
    for i in range(len(handles)):
        if i == 1:
            labels[i] = "670 nm excitation"
        elif i == 5:
            labels[i] = "700 nm excitation"
        else:
            labels[i] = "_Hidden"
    for idx, ax in enumerate(ax_.flatten()):
        ax.set_ylabel(ax.title.get_text().replace("spectral = ", ""))
        if idx > 1:
            ax.set_xlabel("Time (ps)")
        else:
            ax.set_xlabel("")
        ax.set_title("")
        if ax.get_legend() is not None:
            ax.get_legend().remove()
        for line in ax.lines:
            line.set_linewidth(0.5)  # Set the line width here
    fig.legend(
        handles,
        labels,
        bbox_to_anchor=(0.5, -0.05),
        loc="lower center",
        ncol=len(handles),
    )
    fig.tight_layout()

## Overview 670 exc

In [ ]:
# plot_overview(
#     target_result.data["670TR1"],
#     nr_of_data_svd_vectors=4,
#     nr_of_residual_svd_vectors=1,
#     linlog=False,
#     linthresh=1,
#     cycler=cycler(
#         color=["y", "g", "tab:orange", "r", "k", "c", "b", "m", "tab:purple"]
#     ),
# );

## Residual analysis of the 670 nm excitation TR1 data

In [ ]:
import matplotlib.pyplot as plt
from pyglotaran_extras.plotting.plot_residual import plot_residual
from pyglotaran_extras.plotting.plot_svd import plot_lsv_residual
from pyglotaran_extras.plotting.plot_svd import plot_rsv_residual


def plot_residual_and_svd(result_dataset):
    fig, axes = plt.subplots(1, 3, figsize=(10, 2))
    plot_residual(result_dataset, axes[0])
    axes[0].get_legend().remove()
    axes[0].set_ylabel("Wavelength (nm)")
    plot_lsv_residual(result_dataset, axes[1], indices=[0])
    axes[1].get_legend().remove()
    axes[1].set_ylabel("")
    axes[1].set_title("residual 1st LSV")
    plot_rsv_residual(result_dataset, axes[2], indices=[0])
    axes[2].set_xlabel("Wavelength (nm)")
    axes[2].set_title("residual 1st RSV")
    axes[2].get_legend().remove()
    axes[2].set_ylabel("")

    return fig, axes


fig, axes = plot_residual_and_svd(target_result.data["700TR1"])
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)

In [ ]:
# plot_overview(
#     target_result.data["670TR2"],
#     nr_of_data_svd_vectors=4,
#     nr_of_residual_svd_vectors=1,
#     linlog=False,
#     linthresh=1,
#     cycler=cycler(
#         color=["y", "g", "tab:orange", "r", "k", "c", "b", "m", "tab:purple"]
#     ),
# );

## Overview 700 exc

In [ ]:
plot_overview(
    target_result.data["700TR1"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=1,
    linlog=False,
    linthresh=1,
    cycler=cycler(
        color=["y", "g", "tab:orange", "r", "k", "c", "b", "m", "tab:purple"]
    ),
);

In [ ]:
plot_overview(
    target_result.data["700TR2"],
    nr_of_data_svd_vectors=5,
    nr_of_residual_svd_vectors=1,
    linlog=False,
    linthresh=1,
    cycler=cycler(
        color=["y", "g", "tab:orange", "r", "k", "c", "b", "m", "tab:purple"]
    ),
);

# Plot result for interpretation


In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_das
from pyglotaran_extras.plotting.plot_spectra import plot_sas
from pyglotaran_extras.plotting.style import ColorCode

# myFRLcolors = [ "tab:grey","tab:orange",  ColorCode.cyan, ColorCode.green,"m", "y", "k","r", "b", "tab:purple"]
myFRLcolors = ["g","r", "k", ColorCode.cyan,"b"]

custom_cycler = cycler(color=myFRLcolors)
# custom_cycler2 = cycler(color=["k","r", ColorCode.green,"b","m"])
custom_cycler2 = cycler(color=["tab:grey","tab:orange", ColorCode.green,ColorCode.turquoise,"m"])


def plot_concentration_and_spectra(result_dataset,result_dataset2):
    # fig, axes = plt.subplots(1, 2, figsize=(18, 7))
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    plot_concentrations(result_dataset, axes[0], center_λ=0, linlog=True, cycler=custom_cycler)
    plot_concentrations(result_dataset2, axes[0], center_λ=0, linlog=True, cycler=custom_cycler)
    plot_sas(result_dataset, axes[1], cycler=custom_cycler)
    plot_das(result_dataset, axes[2], cycler=custom_cycler2)
    return fig, axes


fig, axes = plot_concentration_and_spectra(target_result.data["700TR1"],target_result.data["700TR2"])
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SADS (mOD)")
axes[1].set_title("SADS")
axes[2].set_xlabel("Wavelength (nm)")
axes[2].set_ylabel("DADS (mOD)")
axes[2].set_title("DADS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

### Amplitude matrices

In [ ]:
show_a_matrixes(target_result)

In [ ]:
compartments = target_scheme.model.initial_concentration["input700"].compartments

target_scheme.model.k_matrix["kmWL"].matrix_as_markdown(compartments).replace("0.0000e+00", "")